# Extending the fastai tabular classification model to handle missing data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sklearn.datasets as sk_data
import sklearn.model_selection as sk_selection
import sklearn.metrics as sk_metrics
import random_neural_net_models.tabular as rnnm_tab
import torch.optim as optim
import random_neural_net_models.losses as rnnm_loss
import random_neural_net_models.learner as rnnm_learner
import random_neural_net_models.data as rnnm_data
import random_neural_net_models.utils as rnnm_utils
from pathlib import Path
from torch.utils.data import DataLoader

In [ ]:
rnnm_utils.make_deterministic(42)

In [ ]:
device = rnnm_utils.get_device()
device

## Generating data

In [ ]:
n_samples = 1_000
n_features = 3
n_classes = 2
X, y = sk_data.make_blobs(
    n_samples=n_samples,
    n_features=n_features,
    random_state=42,
    centers=n_classes,
)

In [ ]:
y = y.astype(int)

In [ ]:
X[:3], y[:3]

In [ ]:
import numpy as np

In [ ]:
cols_with_missingness = (1,)
X_miss, _ = rnnm_tab.make_missing_numerical(
    X, p_missing=0.1, cols_with_missing=cols_with_missingness
)

In [ ]:
np.isfinite(X_miss).all(axis=0)

In [ ]:
X0, X1, y0, y1 = sk_selection.train_test_split(X_miss, y, test_size=0.2)

In [ ]:
ds_train = rnnm_data.NumpyTrainingDataset(X0, y0)
ds_valid = rnnm_data.NumpyTrainingDataset(X1, y1)

In [ ]:
X_miss

In [ ]:
X_miss[0]

In [ ]:
ds_train[2]

In [ ]:
batch_size = 50
dl_train = DataLoader(
    ds_train,
    batch_size=batch_size,
    collate_fn=rnnm_data.collate_numpy_dataset_to_xyblock_keep_orig_y,
)
dl_valid = DataLoader(
    ds_valid,
    batch_size=batch_size,
    collate_fn=rnnm_data.collate_numpy_dataset_to_xyblock_keep_orig_y,
)

In [ ]:
next(iter(dl_train))

In [ ]:
next(iter(dl_train)).x[:10]

## TabularModel

In [ ]:
do_impute = True
impute_bias_source = rnnm_tab.BiasSources.zero

model = rnnm_tab.TabularModel(
    [n_features, 200, 100, n_classes],
    use_batch_norm=False,
    do_impute=do_impute,
    impute_bias_source=impute_bias_source,
    cols_with_missing=cols_with_missingness,
)

In [ ]:
learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = rnnm_loss.CrossEntropyXy()
loss_callback = rnnm_learner.TrainLossCallback()

save_dir = Path("./models")

callbacks = [loss_callback]

In [ ]:
learner = rnnm_learner.Learner(
    model,
    optimizer,
    loss,
    callbacks=callbacks,
    save_dir=save_dir,
    device=device,
    show_epoch_progress=True,
)

In [ ]:
model.net[0].bias.device

In [ ]:
model.net[1].lin.weight.device

In [ ]:
lr_find_callback = rnnm_learner.LRFinderCallback(1e-5, 100, 100)

learner.find_learning_rate(
    dl_train, n_epochs=10, lr_find_callback=lr_find_callback
)

In [ ]:
lr_find_callback.plot(yscale="log")

In [ ]:
learning_rate = 1e-4
n_epochs = 5

scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=learning_rate,
    epochs=n_epochs,
    steps_per_epoch=len(dl_train),
)
scheduler_callback = rnnm_learner.EveryBatchSchedulerCallback(scheduler)
learner.update_callback(scheduler_callback)

In [ ]:
learner.fit(dl_train, n_epochs=n_epochs, dataloader_valid=dl_valid)

In [ ]:
loss_callback.plot()

In [ ]:
X.mean(axis=0), X.std()

In [ ]:
model.net[0].bias

In [ ]:
probs = learner.predict(dl_valid).detach().softmax(dim=1).numpy()[:, 1]
probs[:3]

In [ ]:
y1

In [ ]:
probs

In [ ]:
print(sk_metrics.roc_auc_score(y1, probs))

## TabularModelClassification

In [ ]:
n_hidden = [200, 100]
do_impute = True
impute_bias_source = rnnm_tab.BiasSources.zero

model = rnnm_tab.TabularModelClassification(
    n_features=n_features,
    n_hidden=n_hidden,
    n_classes=n_classes,
    use_batch_norm=False,
    do_impute=do_impute,
    impute_bias_source=impute_bias_source,
    cols_with_missing=cols_with_missingness,
)

In [ ]:
learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = rnnm_loss.CrossEntropyXy()
loss_callback = rnnm_learner.TrainLossCallback()

save_dir = Path("./models")

callbacks = [loss_callback]

In [ ]:
learner = rnnm_learner.Learner(
    model,
    optimizer,
    loss,
    callbacks=callbacks,
    save_dir=save_dir,
    device=device,
    show_epoch_progress=True,
)

In [ ]:
lr_find_callback = rnnm_learner.LRFinderCallback(1e-5, 100, 100)

learner.find_learning_rate(
    dl_train, n_epochs=10, lr_find_callback=lr_find_callback
)

In [ ]:
lr_find_callback.plot(
    yscale="log",
    ylim=(
        0.1,
        1,
    ),
)

In [ ]:
learning_rate = 1e-3
n_epochs = 5

scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=learning_rate,
    epochs=n_epochs,
    steps_per_epoch=len(dl_train),
)
scheduler_callback = rnnm_learner.EveryBatchSchedulerCallback(scheduler)
learner.update_callback(scheduler_callback)

In [ ]:
learner.fit(dl_train, n_epochs=n_epochs, dataloader_valid=dl_valid)

In [ ]:
loss_callback.plot()

In [ ]:
probs = learner.predict(dl_valid).detach().softmax(dim=1).numpy()[:, 1]
probs[:3]

In [ ]:
print(sk_metrics.roc_auc_score(y1, probs))